In [32]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig
import plotly.graph_objects as go


In [33]:
reform = Reform.from_dict({
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

reform_taxable = Reform.from_dict({
  "gov.contrib.states.or.rebate.state_tax_exempt": {
    "2024-01-01.2100-12-31": True
  },
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

In [34]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
reformed_taxable = Microsimulation(reform=reform_taxable)


In [35]:
def calculate_poverty_impact_by_age(baseline, reformed, year):
    state_codes = baseline.calc("state_code", map_to="person", period=year)
    age = baseline.calc("age", map_to="person", period=year)

    baseline_poverty = baseline.calc("in_poverty", map_to="person", period=year)
    reform_poverty = reformed.calc("in_poverty", map_to="person", period=year)

    results = {}
    age_groups = [
        (0, 6, "0-6"),
        (6, 18, "6-18"),
        (18, 65, "18-65"),
        (65, 200, "65+")
    ]

    for min_age, max_age, label in age_groups:
        mask = (state_codes == "OR") & (age >= min_age) & (age < max_age)
        
        baseline_poverty_group = baseline_poverty[mask].mean()
        reform_poverty_group = reform_poverty[mask].mean()
        
        relative_poverty_reduction = (reform_poverty_group - baseline_poverty_group) / baseline_poverty_group
        results[label] = relative_poverty_reduction

    return results

In [36]:
# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame(columns=["year", "age_group", "relative_poverty_reduction", "relative_poverty_reduction_taxable"])


In [37]:
# Calculate for each year and add to the DataFrame
for year in range(2025, 2028):
    results_not_taxable = calculate_poverty_impact_by_age(baseline, reformed, year)
    results_taxable = calculate_poverty_impact_by_age(baseline, reformed_taxable, year)

    for age_group in results_not_taxable.keys():
        new_row = pd.DataFrame({
            "year": [year],
            "age_group": [age_group],
            "relative_poverty_reduction": [results_not_taxable[age_group]],
            "relative_poverty_reduction_taxable": [results_taxable[age_group]]
        })
        all_results = pd.concat([all_results, new_row], ignore_index=True)


In [40]:
# Display the results
print(all_results)

    year age_group  relative_poverty_reduction  \
0   2025       0-6                   -0.136420   
1   2025      6-18                   -0.433609   
2   2025     18-65                   -0.207817   
3   2025       65+                   -0.053603   
4   2026       0-6                   -0.136420   
5   2026      6-18                   -0.482002   
6   2026     18-65                   -0.235611   
7   2026       65+                   -0.050389   
8   2027       0-6                    0.000000   
9   2027      6-18                   -0.482002   
10  2027     18-65                   -0.233850   
11  2027       65+                   -0.076262   

    relative_poverty_reduction_taxable  
0                            -0.136420  
1                            -0.433609  
2                            -0.217906  
3                            -0.053603  
4                            -0.136420  
5                            -0.513606  
6                            -0.243466  
7                    

In [42]:
# Define colors for age groups
colors = {
    "0-6": {"not_taxable": "#0066cc", "taxable": "#333333"},    # Dark blue / Dark grey
    "6-18": {"not_taxable": "#4d94ff", "taxable": "#666666"},   # Medium blue / Medium grey
    "18-65": {"not_taxable": "#99c2ff", "taxable": "#999999"},  # Light blue / Light grey
    "65+": {"not_taxable": "#cce0ff", "taxable": "#cccccc"},    # Very light blue / Very light grey
}

# Create the plot
fig = go.Figure()

# Add traces for each age group
for age_group in colors.keys():
    data = all_results[all_results['age_group'] == age_group]

    # Non-taxable scenario
    fig.add_trace(go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction'],
        mode='lines',
        name=f'{age_group} (Not Taxable)',
        line=dict(color=colors[age_group]['not_taxable'], width=2)
    ))

    # Taxable scenario
    fig.add_trace(go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction_taxable'],
        mode='lines',
        name=f'{age_group} (Taxable)',
        line=dict(
            color=colors[age_group]['taxable'], 
            width=2,
            dash='dot' if age_group in ['0-6', '65+'] else 'solid'  # Dotted line for 0-6 and 65+ taxable
        )
    ))


# Update layout
fig.update_layout(
    title='Oregon Rebate Impact on Poverty by Age Group Over Time',
    xaxis_title='Year',
    yaxis_title='Relative Poverty Reduction (%)',
    legend_title='Age Group and Tax Status',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    ),
    height=600,
    width=800,
)

# Update x-axis to show only three steps
min_year = all_results['year'].min()
max_year = all_results['year'].max()
mid_year = min_year + (max_year - min_year) // 2

fig.update_xaxes(
    tickvals=[min_year, mid_year, max_year],
    ticktext=[str(min_year), str(mid_year), str(max_year)],
    tickformat='d'
)

# Update y-axis
fig.update_yaxes(tickformat='.0%')

# Update hover template
fig.update_traces(
    hovertemplate='Year: %{x}<br>Age Group: %{data.name}<br>Relative Poverty Reduction: %{y:.2%}<extra></extra>'
)

fig = format_fig(fig)
fig.show()